In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests, re, time, urllib3, nltk
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm

## Source 

https://www.citytowninfo.com/places

## State by State Info

In [2]:
url  = 'https://www.citytowninfo.com/places'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
state_dict = {'state':[], 'state_link':[], 'population':[], 'capital':[], 'capital_link':[]}
for row in soup.find('table').find_all('tr')[1:]:
    state_dict['state'].append(row.select_one("td:nth-of-type(1)").text)
    state_dict['state_link'].append(row.select_one("td:nth-of-type(1)").find('a')['href'])
    state_dict['population'].append(row.select_one("td:nth-of-type(2)").text)
    state_dict['capital'].append(row.select_one("td:nth-of-type(3)").text)
    state_dict['capital_link'].append(row.select_one("td:nth-of-type(3)").find('a')['href'])

In [4]:
state_info_dict = {}
for link, state in zip(state_dict['state_link'], state_dict['state']):
    print('{} - {} {}\r'.format(state, link, " "*40), end="")
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.find('div', attrs = {'id':'population_data'}).find_all('table')
    state_info_dict[state] = []
    for table in tables:
        for row in table.find_all('tr'):
            state_info_dict[state].append({row.select_one("td:nth-of-type(1)").text : row.select_one("td:nth-of-type(2)").text})

In [5]:
for state in state_info_dict.keys():
    state_info_dict[state] = dict(map(dict.popitem, state_info_dict[state])) 

In [6]:
state_data = pd.DataFrame(state_info_dict).transpose()
state_data = state_data.dropna(thresh=len(state_data)-1, axis=1).drop(['+ Show More'], axis = 1)

## City By City Data

Here we are grabbing all the links for each of the cities for each of the states

In [7]:
driver = webdriver.Chrome(executable_path ="outputs/chromedriver.exe") 
wait = WebDriverWait(driver, 100)

city_dict = {}
i = 1

for link, state in zip(state_dict['state_link'], state_dict['state']):
    
    print('{} - {}           \r'.format(i, state), end="")
    i += 1
    
    
    driver.get(link) 
    driver.maximize_window()
    driver.implicitly_wait(4)

    city_dict[state] = {
        'city':[],
        'city_link':[]
    }
    
    cities_click = driver.find_element_by_class_name('quidget_article_content').find_element_by_xpath('//*[@id="top-box"]').find_elements_by_class_name('city_alphabet_range')
    for a in cities_click:
        driver.execute_script("arguments[0].click();", a)
        table = driver.find_element_by_xpath('//*[@id="city_list"]')
        bodies = table.find_elements_by_tag_name("tbody") # get all of the rows in the table
        for body in bodies:
            rows = body.find_elements_by_tag_name("tr")
            for row in rows:
                # Get the columns (all the column 2)       
                try:
                    city_dict[state]['city'].append(row.find_elements_by_tag_name("td")[0].text)
                    city_dict[state]['city_link'].append(row.find_element_by_tag_name('a').get_attribute('href'))
                except:
                    pass
driver.close()

## Save as JSON

In [8]:
import json

json_data = json.dumps(city_dict)
f = open("outputs/dict_of_city_links.json","w")
f.write(json_data)
f.close()

In [9]:
with open('outputs/dict_of_city_links.json') as f: 
    city_dict = json.load(f) 

## Get City By City Data

Now we can use the links that we grabbed and scrape the data from each one

In [10]:
city_info_by_state = {}
for state_num, state in enumerate(city_dict.keys()):
    city_info_by_state[state] = {}
    i = 1
    for link, city in zip(city_dict[state]['city_link'], city_dict[state]['city']):
        print('{}) {}: City {} of {} -- {} ({}){}\r'.format(state_num+1, state, i, len(city_dict[state]['city']), city, link, " "*80), end="")
        i += 1
        
        response = requests.get(link)
        soup = BeautifulSoup(response.text, 'html.parser')
        city_info_by_state[state][city] = []
        
        try:
            tables = soup.find('div', attrs = {'id':'population_data'}).find_all('table')
            for table in tables:
                for row in table.find_all('tr'):
                    city_info_by_state[state][city].append({row.select_one("td:nth-of-type(1)").text : row.select_one("td:nth-of-type(2)").text})
        except:
            pass
        
        try:
            tables = soup.find('div', attrs = {'id':'almanac'}).find('table')
            for table in tables:
                for row in table.find_all('tr'):
                    city_info_by_state[state][city].append({row.select_one("td:nth-of-type(1)").text : row.select_one("td:nth-of-type(2)").text})
        except:
            pass
            
        try:
            div = soup.find('div', attrs = {'id':'climate'})
            table = div.findNext('table') # Find the first <table> tag that follows it
            rows = table.findAll('tr')
            for row in rows:
                city_info_by_state[state][city].append({row.select_one("td:nth-of-type(1)").text : row.select_one("td:nth-of-type(2)").text})
        except:
            pass
        
for state in city_info_by_state.keys():
    for city in city_info_by_state[state].keys():
        city_info_by_state[state][city] = dict(map(dict.popitem, city_info_by_state[state][city])) 

## Save as JSON

In [11]:
import json

json_data = json.dumps(city_info_by_state)
f = open("outputs/dict_of_city_data.json","w")
f.write(json_data)
f.close()

In [12]:
with open('outputs/dict_of_city_data.json') as f: 
    city_info_by_state = json.load(f) 

## Convert Dict to DataFrame

In [13]:
city_data = pd.DataFrame.from_dict(
    {
        (i,j): city_info_by_state[i][j] 
        for i in city_info_by_state.keys() 
        for j in city_info_by_state[i].keys()
    }, 
    orient='index'
)
percent_to_keep = .8
city_data = city_data.dropna(thresh=int(percent_to_keep*len(city_data)), axis = 1).drop(['+ Show More', 'Station', 'Distance', ' Category'], axis = 1).dropna(subset=['Total Population'])

## Save as CSV

In [14]:
city_data.to_csv('outputs/all_city_data.csv')

## Load and Transform Raw CSV

In [15]:
city_data = pd.read_csv('outputs/all_city_data.csv')
city_data = city_data.rename(columns={'Unnamed: 0': 'State', 'Unnamed: 1': 'City'})
city_data = city_data.replace({'%': '', '\$': '', ',':''}, regex=True).replace({r'^\s*$':np.nan, 'nan':np.nan}, regex=True)

There a lot of extra symbols in the data like % and \$ so I took those out

In [16]:
for col in [column for column in city_data.drop(['Time Zone', 'State', 'City'], axis=1).columns if city_data[column].dtype == 'object']:
    test_list = []
    for row in city_data[col]:
        try:
            number = re.findall(r"(?<![a-zA-Z:])[-+]?\d*\.?\d+", row)
            if len(number) == 1:
                test_list.append(number[0])
            else:
                test_list.append(number)
        except:
            test_list.append(np.nan)
    city_data[col] = test_list

Now I parsed the numbers from each of the cells, expect those that were obviously objects. Some had more than 1 number so those went into lists

In [17]:
#city_data[['Oct Temp','Jul Temp','Apr Temp','Jan Temp']] 
for column, season in zip(['Oct Temp','Jul Temp','Apr Temp','Jan Temp'], ['Fall','Summer','Spring','Winter']):
    low = []
    high = []
    for row in city_data[column]:
        if str(type(row)) == "<class 'float'>":
            low.append(np.nan)
            high.append(np.nan)
        else:
            low.append(row[0])
            high.append(row[1])
    names = [season + ' Low', season + ' High']
    city_data[names[0]] = low
    city_data[names[1]] = high

We can now brake apart those lists. It turns out it was only the temps (high and low). 

In [18]:
city_data = city_data.convert_objects(convert_numeric=True).drop(['Oct Temp','Jul Temp','Apr Temp','Jan Temp'], axis = 1)

C:\Users\calla\Anaconda3\envs\pyMain\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


It took a lot of work, but now I can convert the numbers from object to floats and ints. Columns that had NaNs were assigned to floats.

In [19]:
percentage_columns = ['Male Share of the Population','Female Share of the Population', 'Senior Citizens',
       '% of people married','Population % with Bachelor Degree or Higher', 
       '% Above Poverty Level', '% Below Poverty Level', '% Working from Home',
       '% Walking and Biking to Work', '% Using Public Transportation','People Living Alone',
       'Other(often includes Hispanic and African American)', 'German',
       'Irish', 'French Except Basque', 'Scottish', 'Italian', 'Dutch','English']

for key, value in city_data[percentage_columns].iteritems():
    test_list = []
    for x in value:
        test_list.append(x/100)
    city_data[key] = test_list

Finally, I converted the percentages from number between 1-100 to 0-1 by dividing those numbers by 100

## Resave Clean City Data

In [20]:
city_data.to_csv('static/data/all_city_data_clean.csv', index=False)